In [1]:
import pandas as pd
import numpy as np
import os
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
import sys
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/My Drive/Colab Notebooks/OSU/CS467_shared'

/content/drive/.shortcut-targets-by-id/1Y2mRkkRB6wyTJ1-1zFboT6i8X4wUbIq6/CS467_shared


In [4]:
IMG_PIXELS = 67000
IMG_WIDTH = 335
IMG_HEIGHT = 200
NUM_LABELS = 32
NUM_FEATURES = 45

In [5]:
FMA_images_dataset = list(tf.data.experimental.load('./Data/FMA_images_dataset',(tf.TensorSpec(shape=(IMG_PIXELS,), dtype=tf.uint8, name=None), tf.TensorSpec(shape=(NUM_LABELS,), dtype=tf.int8, name=None))))
FMA_features_dataset = list(tf.data.experimental.load('./Data/FMA_features_dataset',(tf.TensorSpec(shape=(NUM_FEATURES,), dtype=tf.float32, name=None), tf.TensorSpec(shape=(NUM_LABELS,), dtype=tf.int8, name=None))))
labels_key = pd.read_csv('./Data/labels_key.csv')['category']

In [6]:
images = []
labels = []
features = []
track_ids = []

for i, (x, y) in enumerate(FMA_images_dataset):
  images.append(x.numpy())
  features.append(FMA_features_dataset[i][0][1:].numpy())
  labels.append(y.numpy())
  track_ids.append(FMA_features_dataset[i][0][0].numpy())

images = np.array(images)
labels = np.array(labels)
features = np.array(features)
track_ids = np.array(track_ids)

FMA_images_dataset = None
FMA_features_dataset = None

In [7]:
print(len(images))
print(len(features))
print(len(labels))
print(len(track_ids))

100817
100817
100817
100817


In [8]:
print(features[0])

[ 3.9690730e-01  2.4936402e-03  1.0000000e+00  2.9430017e-01
  1.4519466e-01  2.1372719e-02  4.0085819e-01  7.8943834e-02
  3.0376462e+03  8.2565656e+02  6.8770391e+03  9.9045245e+02
  2.6901113e+03  1.5655203e+03  3.7333093e+03  3.5741660e+02
  6.2146641e+03  1.5180908e+03  9.6468750e+03  1.5103333e+03
  1.6371983e-01  2.9296875e-02  6.1181641e-01  9.3396619e-02
 -6.7384224e+01  6.5154984e+01 -1.0669198e+01  1.2176895e+01
 -3.3035460e+00  9.9164639e+00 -1.9857939e+00  5.4153090e+00
 -6.1618333e+00  1.0913678e+00 -1.0330084e+01  3.4071732e+00
 -3.0144753e+00  8.1967850e+00 -3.3063750e+00  1.2533796e+00
 -8.4815273e+00 -2.0407531e+00 -2.9466519e+00  9.4058223e-02]


In [9]:
model = tf.keras.models.load_model('FMA_model_seperate_genres.h5')

In [10]:
images = images.reshape(images.shape[0], IMG_HEIGHT, IMG_WIDTH, 1)
features = StandardScaler().fit_transform(features)

In [11]:
top5_count = 0
top1_count = 0
cur_score = 0
highest_score_possible = 0

for n, label in enumerate(labels):
  num_genres = np.count_nonzero(label)
  genre_weight = 1/num_genres
  highest_score_possible += 5

  # add indicies of track label to label_index array
  label_index = []
  for i, v in enumerate(label):
    if v == 1:
      label_index.append(i)

  # top 5 predictions
  prediction = model.predict([np.array([features[n]]), np.array([images[n]])])
  topN_index = []
  topN = np.argsort(-prediction[0])[:5]
  for i, val in enumerate(topN, start=1):
    topN_index.append(val)

  # compare label indicies to determine if prediction includes correct label
  # top1
  for i in label_index:
    if i == topN_index[0]:
      top1_count += 1
  # top5
  for i in topN_index:
    if i in label_index:
      top5_count += 1
      break

  # calculate per-track genre score and add to total model score 
  for j, i in enumerate(topN_index):
    if i in label_index:
      # if top most genre is correct and the track only has a single genre 
      # the formula doesnt work so we handle that case independenly
      if i == label_index[0] and num_genres == 1:
        genre_score = 5
        cur_score += genre_score
        break
      else:
        genre_score = genre_weight * (5-j) + genre_weight
        # print('genre_score: ', genre_score)
        cur_score += genre_score
        if genre_score == 0:
          print('genre not found')


  if n > 0 and n % 10000 == 0:
    print('''sample: {n} \n
          top1_count: {top1_count} \n
          top1 accuracy: {top1_acc} \n
          top5_count: {top5_count} \n
          top5 accuracy: {top5_acc} \n
          total score: {total_score} \n
          score: {score} \n'''.format(
                      n=n, top1_count=top1_count, top1_acc=top1_count/n, 
                      top5_count=top5_count, top5_acc=top5_count/n,
                      total_score=cur_score, 
                      score=cur_score/highest_score_possible))

total_n = len(labels)
top1_acc = top1_count / total_n
top5_count = top5_count
top5_acc = top5_count / total_n
total_score = cur_score
score = cur_score / highest_score_possible

sample: 10000 

          top1_count: 4956 

          top1 accuracy: 0.4956 

          top5_count: 7580 

          top5 accuracy: 0.758 

          total score: 32442.266666666845 

          score: 0.6487804552878081 

sample: 20000 

          top1_count: 10248 

          top1 accuracy: 0.5124 

          top5_count: 15286 

          top5 accuracy: 0.7643 

          total score: 63388.17222222129 

          score: 0.6338500297207269 

sample: 30000 

          top1_count: 15176 

          top1 accuracy: 0.5058666666666667 

          top5_count: 22640 

          top5 accuracy: 0.7546666666666667 

          total score: 91305.54700577332 

          score: 0.6086833572599134 

sample: 40000 

          top1_count: 20108 

          top1 accuracy: 0.5027 

          top5_count: 30193 

          top5 accuracy: 0.754825 

          total score: 121263.83033910858 

          score: 0.6063039940956905 

sample: 50000 

          top1_count: 25622 

          top1 accuracy: 0.51

In [12]:
print("total_n: ", total_n)
print("top1_acc: ", top1_acc)
print("top5_count: ", top5_count)
print("top5_acc: ", top5_acc)
print("total_score: ", total_score)
print("score: ", score)

total_n:  100817
top1_acc:  0.5163811658748029
top5_count:  78084
top5_acc:  0.7745122350397254
total_score:  298586.1339105312
score:  0.5923329079630046
